In [1]:
import tensorflow as tf
import numpy as np
import common.utils as utils
from lstm.train import construct_graph

num_steps = 100
max_length = 200

def update_feed():
  input = np.zeros([1, num_steps], dtype=np.int64)
  dictionary = utils.load_dictionary()
  input[b, 0] = dictionary['<bof>']
  return input

def sample(prediction):
  return np.random.choice(prediction.size, (), p=prediction)

def format_feed(input):
  """Format input to feed to LSTM
  Implicitly batch_size = 1
  """
  length = len(input)
  if length < num_steps:
    return [input + [0]*(num_steps-length)]
  else:
    return [input[-num_steps:]]


In [2]:
graph = tf.Graph()
optimizer, loss, inputs, probs, learning_rate, dictionary, saver, summary = construct_graph(graph)
with tf.Session(graph=graph) as sess:
  saver.restore(sess, "checkpoints/model.ckpt")
  coord = tf.train.Coordinator()
  threads = tf.train.start_queue_runners(sess=sess, coord=coord)

  dictionary = utils.load_dictionary()
  feed_input = []
  sample_character = dictionary['<bof>']
  feed_input.append(sample_character) # begin with <bof>
  try:
    for i in range(1):
      feed_dict = {inputs: format_feed(feed_input)} # feed last <num_steps> words
      test_probs = probs.eval(feed_dict=feed_dict) # predict
      sample_character = sample(test_probs) # sampling
      feed_input.append(sample_character) # update feed
      if sample_character == '<eof>': # stop sampling when meet <eof>
        break
  except Exception as e:
    print('Exception: {}'.format(e.args))
    coord.request_stop(e)

  print(feed_input)
  # utils.print_data(feed_input, pretty=True)

  coord.request_stop()
  coord.join(threads)

[1, array(5775)]


In [11]:
test_probs.shape

(100, 20368)

In [30]:
sample(test_probs[0])[()]

17821

In [27]:
print(s)

16365
